# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

print('WORKSPACE DETAILS:\nWorkspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

# choose a name for experiment
experiment_name = 'Heart_Disease_Kaggle_AutoML'

experiment=Experiment(ws, experiment_name)
print("Experiment created")

run = experiment.start_logging()

WORKSPACE DETAILS:
Workspace name: quick-starts-ws-136809
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-136809
Experiment created


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Creating a compute cluster
# vm_size = "Standard_D2_V2"
# max_nodes = 4.

cpu_cluster_name = "cpu-cluster"

# to check whether the compute cluster exists already or not
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Existing compute target found... Using it")

except ComputeTargetException:
    print("Creating new Compute Target... " + cpu_cluster_name)
    provisioning_compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, provisioning_compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Existing compute target found... Using it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Exploring the Dataset

In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd

url_path = "https://raw.githubusercontent.com/bharati-21/AZMLND-Capstone-Project/master/files/heart.csv"
ds = TabularDatasetFactory.from_delimited_files(path=url_path)

df = pd.read_csv(url_path)

In [7]:
ds_dataframe = ds.to_pandas_dataframe()
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x.head()

Index(['age', 'resting_BP', 'cholesterol', 'max_heart_rate', 'st_depression',
       'num_major_vessels', 'target', 'sex_Male',
       'chest_pain_yype_atypical_angina', 'chest_pain_yype_non-anginal pain',
       'chest_pain_yype_typical_angina', 'fasting_blood_sugar_less_than_120',
       'rest_ECG_left_ventricular_hypertrophy', 'rest_ECG_normal',
       'exercise_induced_angina_Yes', 'st_slope_flat', 'st_slope_upsloping',
       'thalassemia_normal', 'thalassemia_reversible_defect'],
      dtype='object')
(302, 19)


,age,resting_BP,cholesterol,max_heart_rate,st_depression,num_major_vessels,sex_Male,chest_pain_yype_atypical_angina,chest_pain_yype_non-anginal pain,chest_pain_yype_typical_angina,fasting_blood_sugar_less_than_120,rest_ECG_left_ventricular_hypertrophy,rest_ECG_normal,exercise_induced_angina_Yes,st_slope_flat,st_slope_upsloping,thalassemia_normal,thalassemia_reversible_defect
0,63,145,233,150,2.3,0.0,1,0,0,0,0,0,1,0,0,1,0,0
1,37,130,250,187,3.5,0.0,1,0,1,0,1,0,0,0,0,1,0,1
2,41,130,204,172,1.4,0.0,0,1,0,0,1,0,1,0,0,0,0,1
3,56,120,236,178,0.8,0.0,1,1,0,0,1,0,0,0,0,0,0,1
4,57,120,354,163,0.6,0.0,0,0,0,1,1,0,0,1,0,0,0,1


In [9]:
import os
path = "./Data"
try:
    os.makedirs(path, exist_ok=True)
    print("Dicrectory 'Data' created...")
except OSError:
    print("Directory 'Data' cannot be created...")

Dicrectory 'Data' created...


In [10]:
# split data into test and train sets

from sklearn.model_selection import train_test_split
import pandas as pd

#spliting the data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2)
print(test_x.shape, train_x.shape, test_y.shape, train_y.shape)

train_df = train_x.copy(deep=True)
train_df['target'] = train_y
print(train_df.shape)

test_df = test_x.copy(deep=True)
test_df['target'] = test_y
print(test_df.shape)

# saving test and train data locally
train_df.to_csv("./Data/train_data.csv")
test_df.to_csv("./Data/test_data.csv")

(61, 18) (241, 18) (61,) (241,)
(241, 19)
(61, 19)


In [11]:
deafult_store = ws.get_default_datastore()
deafult_store.upload(src_dir="Data", target_path="Data")

Uploading an estimated of 2 files
Target already exists. Skipping upload for Data/test_data.csv
Target already exists. Skipping upload for Data/train_data.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_bd3fa28630b94dda99cd7e8371ee0249

In [12]:
from azureml.core import Dataset

train_data = TabularDatasetFactory.from_delimited_files(path=[(deafult_store, ('Data/train_data.csv'))])
test_data = TabularDatasetFactory.from_delimited_files(path=[(deafult_store, ('Data/test_data.csv'))])


In [13]:
train_data.to_pandas_dataframe()

,Column1,age,resting_BP,cholesterol,max_heart_rate,st_depression,num_major_vessels,sex_Male,chest_pain_yype_atypical_angina,chest_pain_yype_non-anginal pain,chest_pain_yype_typical_angina,fasting_blood_sugar_less_than_120,rest_ECG_left_ventricular_hypertrophy,rest_ECG_normal,exercise_induced_angina_Yes,st_slope_flat,st_slope_upsloping,thalassemia_normal,thalassemia_reversible_defect,target
0,279,61,138,166,125,3.6,1.0,1,0,0,1,1,0,1,1,1,0,0,1,0
1,76,51,125,245,166,2.4,0.0,1,0,1,0,0,0,1,0,1,0,0,1,1
2,126,47,112,204,143,0.1,0.0,1,0,0,1,1,0,0,0,0,0,0,1,1
3,112,64,140,313,133,0.2,0.0,0,0,1,0,1,0,0,0,0,0,1,0,1
4,290,61,148,203,161,0.0,1.0,1,0,0,1,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,142,42,120,209,173,0.0,0.0,0,0,1,0,1,0,0,0,1,0,0,1,1
207,57,45,115,260,185,0.0,0.0,1,0,0,1,1,0,1,0,0,0,0,1,1
208,51,66,120,302,151,0.4,0.0,1,0,0,1,1,0,1,0,1,0,0,1,1
209,178,43,120,177,120,2.5,0.0,1,0,0,1,1,0,1,1,1,0,1,0,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here

automl_config = AutoMLConfig (
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name="target",
    n_cross_validations=5,
    enable_onnx_compatible_models=True,
    compute_target = cpu_cluster)

print("AutoML config created.")

AutoML config created.


In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_5a3fea9e-2065-475c-a445-ba63e662354f

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)

In [ ]:
print(fitted_model)

In [ ]:
best_run.get_tags()

In [ ]:
import joblib
from azureml.core.model import Model

#TODO: Save the best model

description = "Best Model from AutoML run of the Cleveland Foundation Clinic Heart Disease Dataset"
automl_model = remote_run.register_model(model_name='heart_disease_autoML_best_model.pkl', description=description)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service